In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import torch.nn as nn

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), #input -? OUtput? RF
                     nn.BatchNorm2d(32))
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),
                     nn.BatchNorm2d(64))
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
                     nn.BatchNorm2d(128))
        self.conv4 = nn.Sequential(nn.Conv2d(128, 256, 3, padding=1),
                     nn.BatchNorm2d(256))
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Sequential(nn.Conv2d(256, 512, 3),
                     nn.BatchNorm2d(512))
        self.conv6 = nn.Sequential(nn.Conv2d(512, 1024, 3),
                     nn.BatchNorm2d(1024))
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        # x = F.BatchNorm2d(32)
        x = self.conv2(x)
        x = F.relu(x)
        # x = F.BatchNorm2d(32)
        # F.relu(self.conv2(F.relu(self.conv1(x))))
        x = self.pool1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool2(x)
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.conv5(x)
        x = F.relu(x)
        x = self.conv6(x)
        x = F.relu(x)
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)#dropped activation function in last layer for more accuracy
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
       BatchNorm2d-4           [-1, 64, 28, 28]             128
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
       BatchNorm2d-7          [-1, 128, 14, 14]             256
            Conv2d-8          [-1, 256, 14, 14]         295,168
       BatchNorm2d-9          [-1, 256, 14, 14]             512
        MaxPool2d-10            [-1, 256, 7, 7]               0
           Conv2d-11            [-1, 512, 5, 5]       1,180,160
      BatchNorm2d-12            [-1, 512, 5, 5]           1,024
           Conv2d-13           [-1, 1024, 3, 3]       4,719,616
      BatchNorm2d-14           [-1, 102

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.037598807364702225 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.69it/s]



Test set: Average loss: 0.0450, Accuracy: 9845/10000 (98%)



loss=0.002355577191337943 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.69it/s]



Test set: Average loss: 0.0252, Accuracy: 9930/10000 (99%)



loss=0.0032280338928103447 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.63it/s]



Test set: Average loss: 0.0238, Accuracy: 9923/10000 (99%)



loss=0.0014161948347464204 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.61it/s]



Test set: Average loss: 0.0208, Accuracy: 9936/10000 (99%)



loss=0.00010173358168685809 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.60it/s]



Test set: Average loss: 0.0172, Accuracy: 9951/10000 (100%)

